# Feature Engineering

# Reading Dataset

In [3]:
import pandas as pd

master_df = pd.read_csv("master_pds_dataset.csv")

print(master_df.shape)
master_df.head()

(499494, 52)


,distCode,distName_x,officeCode_x,officeName_x,shopNo,month,year,rcNfsaAay,unitsNfsaAay,rcNfsaPhh,...,otherShopTransCnt,distName,officeCode,officeName,address,longitude,latitude,fpsStatus,fpsType,dateTime
0,532,Adilabad,532001,Talamadugu,1901001,7,2024,38,125,920,...,36,Adilabad,532001.0,Talamadugu,DR Depo TALAMADUGU Talamadugu,78.391212,19.641643,Active,Normal Shop,2025-07-03 14:43:54.124224+05:30
1,532,Adilabad,532001,Talamadugu,1901002,7,2024,20,70,258,...,28,Adilabad,532001.0,Talamadugu,Sunkari Ramesh Sunkidi,78.422012,19.669093,Active,Normal Shop,2025-07-03 14:43:54.124224+05:30
2,532,Adilabad,532001,Talamadugu,1901003,7,2024,17,60,158,...,22,Adilabad,532001.0,Talamadugu,Sakipelli Gajanan Umdam,78.427186,19.664645,Active,Normal Shop,2025-07-03 14:43:54.124224+05:30
3,532,Adilabad,532001,Talamadugu,1901004,7,2024,14,45,190,...,12,Adilabad,532001.0,Talamadugu,Rebbathi Rakesh Lingi,78.381016,19.696032,Active,Normal Shop,2025-07-03 14:43:54.124224+05:30
4,532,Adilabad,532001,Talamadugu,1901005,7,2024,155,520,530,...,30,Adilabad,532001.0,Talamadugu,T Manohar Kuchulapur,78.351118,19.697063,Active,Normal Shop,2025-07-03 14:43:54.124224+05:30


## 🔥 Step 1: Drop Useless Columns

In [5]:
cols_to_drop = [
    "mroAsoApprDate",
    "cardTypeId",
    "cardPoolType"
]

master_df.drop(columns=cols_to_drop, inplace=True)

## 🔥 Step 2: Create Core Behavioral Features

### 1️⃣ Utilization Ratio

In [6]:
master_df["utilization_ratio"] = (
    master_df["noOfTrans"] / master_df["totalRcs"])

### 2️⃣ Total Rice Distributed

In [7]:
master_df["total_rice"] = (
    master_df["riceAfsc"] +
    master_df["riceFsc"] +
    master_df["riceAap"]
)

### 3️⃣ Commodity Intensity (Rice to Wheat)

In [8]:
master_df["rice_wheat_ratio"] = (
    master_df["total_rice"] /
    (master_df["wheat"] + 1)   # avoid division by zero
)

### 4️⃣ Portability Ratio

In [9]:
master_df["portability_ratio"] = (
    master_df["otherShopTransCnt"] /
    (master_df["noOfTrans"] + 1)
)

### Average Monthly Transactions per Shop

In [14]:
avg_trans = (
    master_df
    .groupby(["distCode", "shopNo"])["noOfTrans"]
    .mean()
    .reset_index()
    .rename(columns={"noOfTrans": "avg_monthly_transactions"})
)

master_df = master_df.merge(
    avg_trans,
    on=["distCode", "shopNo"],
    how="left"
)

## 🔥 Step 3: Volatility Feature

In [13]:
# Step 1 — Aggregate Yearly Transactions per Shop

yearly_trans = (
    master_df
    .groupby(["distCode", "shopNo", "year"])["noOfTrans"]
    .sum()
    .reset_index()
)

# Step 2 — Compute Std Across Years
volatility_df = (
    yearly_trans
    .groupby(["distCode", "shopNo"])["noOfTrans"]
    .std()
    .reset_index()
    .rename(columns={"noOfTrans": "yearly_transaction_volatility"})
)

# Step 3 — Merge Back

master_df = master_df.merge(
    volatility_df,
    on=["distCode", "shopNo"],
    how="left"
)


In [15]:
master_df.to_csv("master_pds_dataset.csv", index=False)
print("Master dataset saved successfully.")

Master dataset saved successfully.
